## Generate dataset

In [1]:
## pip install opencv-python
import cv2

In [2]:
def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    def face_cropped(img):
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        # scaling factor = 1.3
        # minimum neighbor = 5
        
        if faces is ():
            return None
        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h,x:x+w]
        return cropped_face
    
    cap = cv2.VideoCapture(1)
    id =3
    img_id = 0
    
    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id+=1
            face = cv2.resize(face_cropped(frame), (200,200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = "data/user."+str(id)+"."+str(img_id)+".jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            
            cv2.imshow("Cropped face", face)
            
        if cv2.waitKey(1)==13 or int(img_id)==200: #13 is the ASCII character of Enter
            break
            
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed....")
#generate_dataset()

## Train the classifier and save it

In [3]:
import os
import cv2
from PIL import Image #pip install pillow
import numpy as np    # pip install numpy

In [4]:
def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    
    faces = []
    ids = []
    
    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
        
        faces.append(imageNp)
        ids.append(id)
        
    ids = np.array(ids)
    
    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
train_classifier("data")

## Detect the faces and give the name if it is already stored in dataset

In [1]:
import cv2
import numpy as np
from PIL import Image
import os

In [6]:
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    
    for (x,y,w,h) in features:
        cv2.rectangle(img, (x,y), (x+w,y+h), color, 2 )
        
        id, pred = clf.predict(gray_img[y:y+h,x:x+w])
        confidence = int(100*(1-pred/300))
        
        if confidence>70:
            if id==1:
                cv2.putText(img, "Ishwar", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            if id==2:
                cv2.putText(img, "Manish", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, "UNKNOWN", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)
    
    return img

# loading classifier
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

video_capture = cv2.VideoCapture(1)

while True:
    ret, img = video_capture.read()
    img = draw_boundary(img, faceCascade, 1.3, 6, (255,255,255), "Face", clf)
    cv2.imshow("face Detection", img)
    
    if cv2.waitKey(1)==13:
        break
video_capture.release()
cv2.destroyAllWindows()


# GUI face recognizer

In [3]:
import tkinter as tk
from tkinter import messagebox

In [13]:
window = tk.Tk()
window.title("Face Recognition system")

l1 = tk.Label(window, text="Name", font=("Algerian",20))
l1.grid(column=0, row=0)
t1 = tk.Entry(window, width=50, bd=5)
t1.grid(column=1, row=0)

l2 = tk.Label(window, text="Age", font=("Algerian",20))
l2.grid(column=0, row=1)
t2 = tk.Entry(window, width=50, bd=5)
t2.grid(column=1, row=1)

l3 = tk.Label(window, text="Address", font=("Algerian",20))
l3.grid(column=0, row=2)
t3 = tk.Entry(window, width=50, bd=5)
t3.grid(column=1, row=2)


b1 = tk.Button(window, text="Training", font=("Algerian",20),bg="orange",fg="red")
b1.grid(column=0, row=4)

b2 = tk.Button(window, text="Detect the faces", font=("Algerian",20), bg="green", fg="orange")
b2.grid(column=1, row=4)

b3 = tk.Button(window, text="Generate dataset", font=("Algerian",20), bg="pink", fg="black")
b3.grid(column=2, row=4)

window.geometry("800x200")
window.mainloop()

## Connecting to the database

In [1]:
import cv2
import os
from PIL import Image
import numpy as np
import mysql.connector  # pip install mysql-connector

In [8]:
window = tk.Tk()
window.title("Face Recognition system")
window.resizable(0,0)

l1 = tk.Label(window, text="Name", font=("Algerian",20))
l1.grid(column=0, row=0)
t1 = tk.Entry(window, width=50, bd=5)
t1.grid(column=1, row=0)

l2 = tk.Label(window, text="Age", font=("Algerian",20))
l2.grid(column=0, row=1)
t2 = tk.Entry(window, width=50, bd=5)
t2.grid(column=1, row=1)

l3 = tk.Label(window, text="Address", font=("Algerian",20))
l3.grid(column=0, row=2)
t3 = tk.Entry(window, width=50, bd=5)
t3.grid(column=1, row=2)

def train_classifier():
    data_dir = "C:/Users/Ishwar Gautam/Desktop/Face Recognition System/data"
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    
    faces = []
    ids = []
    
    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
        
        faces.append(imageNp)
        ids.append(id)
        
    ids = np.array(ids)
    
    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
    messagebox.showinfo("Result","Model trained completed.....")
    
b1 = tk.Button(window, text="Training", font=("Algerian",20),bg="orange",fg="red", command=train_classifier)
b1.grid(column=0, row=4)

def detect_face():
    
    def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)

        for (x,y,w,h) in features:
            cv2.rectangle(img, (x,y), (x+w,y+h), color, 2 )

            id, pred = clf.predict(gray_img[y:y+h,x:x+w])
            confidence = int(100*(1-pred/300))
            
            mydb = mysql.connector.connect(
            host = "localhost",
            user = "root",
            passwd = "",
            database = "Authorized_user"
            )
            mycursor = mydb.cursor()
            
            mycursor.execute("select name from my_table where id=" + str(id))
            s = mycursor.fetchone() #tuple
            
            s  = ''+''.join(s) #string
            
            if confidence>70:
                cv2.putText(img, s , (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            else:
                cv2.putText(img, "UNKNOWN", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)

        return img

    # loading classifier
    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")

    video_capture = cv2.VideoCapture(1)

    while True:
        ret, img = video_capture.read()
        img = draw_boundary(img, faceCascade, 1.3, 6, (255,255,255), "Face", clf)
        cv2.imshow("face Detection", img)

        if cv2.waitKey(1)==13:
            break
    video_capture.release()
    cv2.destroyAllWindows()

b2 = tk.Button(window, text="Detect the faces", font=("Algerian",20), bg="green", fg="orange", command=detect_face)
b2.grid(column=1, row=4)

def generate_dataset():
    if (t1.get()=="" or t2.get()=="" or t3.get()==""):
        messagebox.showinfo("Result","Please provide complete details of the user...")
    else:
        mydb = mysql.connector.connect(
            host = "localhost",
            user = "root",
            passwd = "",
            database = "Authorized_user"
        )
        mycursor = mydb.cursor()
        mycursor.execute("SELECT * from my_table")
        myresult = mycursor.fetchall()
        
        id = 1
        for x in myresult:
            id +=1
        
        sql = "insert into my_table(id,name,age,address) values(%s, %s, %s, %s)"
        val = (id, t1.get(), t2.get(), t3.get())
        mycursor.execute(sql,val)
        mydb.commit()
        
        face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
        def face_cropped(img):
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray, 1.3, 5)
            # scaling factor = 1.3
            # minimum neighbor = 5

            if faces is ():
                return None
            for (x,y,w,h) in faces:
                cropped_face = img[y:y+h,x:x+w]
            return cropped_face

        cap = cv2.VideoCapture(1)
        img_id = 0

        while True:
            ret, frame = cap.read()
            if face_cropped(frame) is not None:
                img_id+=1
                face = cv2.resize(face_cropped(frame), (200,200))
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                file_name_path = "data/user."+str(id)+"."+str(img_id)+".jpg"
                cv2.imwrite(file_name_path, face)
                cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)

                cv2.imshow("Cropped face", face)

            if cv2.waitKey(1)==13 or int(img_id)==200: #13 is the ASCII character of Enter
                break

        cap.release()
        cv2.destroyAllWindows()
        messagebox.showinfo("Result","Generating dataset is completed.....")

b3 = tk.Button(window, text="Generate dataset", font=("Algerian",20), bg="pink", fg="black", command=generate_dataset)
b3.grid(column=2, row=4)

window.geometry("800x200")
window.mainloop()

## Final GUI Face Recognition system using Tkinter canvas

In [1]:
import tkinter as tk
from tkinter import *
import cv2
import os
from PIL import Image, ImageTk
import numpy as np
import mysql.connector
from tkinter import messagebox

##### Divide window

In [3]:
window = tk.Tk()
window.title("Project")
window.resizable(0,0)

load1 = Image.open("header.png")
photo1 = ImageTk.PhotoImage(load1)


header = tk.Button(window, image=photo1)
header.place(x=5, y=0)

canvas1 = Canvas(window, width = 500, height=250, bg='ivory')
canvas1.place(x=5, y=120)

l1 = tk.Button(canvas1, text="Name", font=("Algerian",15))
l1.place(x=5, y=5)
t1 = tk.Entry(canvas1, width=50, bd=5)
t1.place(x=150, y=10)

l2 = tk.Button(canvas1, text="Age", font=("Algerian",15))
l2.place(x=5, y=50)
t2 = tk.Entry(canvas1, width=50, bd=5)
t2.place(x=150, y=55)

l3 = tk.Button(canvas1, text="Address", font=("Algerian",15))
l3.place(x=5, y=100)
t3 = tk.Entry(canvas1, width=50, bd=5)
t3.place(x=150, y=105)


b1= tk.Button(canvas1, text="Training", font=("Algerian",20), bg='orange', fg='red')
b1.place(x=10, y=180)

b2= tk.Button(canvas1, text="Generate dataset", font=("Algerian",20), bg='pink', fg='black')
b2.place(x=175, y=180)

load2= Image.open("canvas2.png")
photo2 = ImageTk.PhotoImage(load2)

canvas2 =  Canvas(window, width=500, height=250)
canvas2.place(x=5, y=400)
canvas2.create_image(250,125,image=photo2)

b3 = tk.Button(canvas2, text="Capture the image", font=("Algerian",20), bg="gray", fg="black")
b3.place(x=5, y=50)

b4 = tk.Button(canvas2, text="Predict face from live video", font=("Algerian", 20), bg="cyan", fg="black")
b4.place(x=5, y=150)

load3= Image.open('canvas3.png')
photo3 = ImageTk.PhotoImage(load3)

canvas3 = Canvas(window, width=280, height=530)
canvas3.place(x=515, y=120)
canvas3.create_image(140, 265, image=photo3)

window.geometry("800x680")
window.mainloop()

# Here is the final code.....

In [9]:
window = tk.Tk()
window.title("Project")
window.resizable(0,0)

load1 = Image.open("header.png")
photo1 = ImageTk.PhotoImage(load1)


header = tk.Button(window, image=photo1)
header.place(x=5, y=0)

canvas1 = Canvas(window, width = 500, height=250, bg='ivory')
canvas1.place(x=5, y=120)

l1 = tk.Button(canvas1, text="Name", font=("Algerian",15))
l1.place(x=5, y=5)
t1 = tk.Entry(canvas1, width=50, bd=5)
t1.place(x=150, y=10)

l2 = tk.Button(canvas1, text="Age", font=("Algerian",15))
l2.place(x=5, y=50)
t2 = tk.Entry(canvas1, width=50, bd=5)
t2.place(x=150, y=55)

l3 = tk.Button(canvas1, text="Address", font=("Algerian",15))
l3.place(x=5, y=100)
t3 = tk.Entry(canvas1, width=50, bd=5)
t3.place(x=150, y=105)

def train_classifier():
    data_dir = "C:/Users/Ishwar Gautam/Desktop/Face Recognition System/data"
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    
    faces = []
    ids = []
    
    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
        
        faces.append(imageNp)
        ids.append(id)
        
    ids = np.array(ids)
    
    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
    messagebox.showinfo("Result","Model trained completed.....")
    

b1= tk.Button(canvas1, text="Training", font=("Algerian",20), bg='orange', fg='red', command=train_classifier)
b1.place(x=10, y=180)


def generate_dataset():
    if (t1.get()=="" or t2.get()=="" or t3.get()==""):
        messagebox.showinfo("Result","Please provide complete details of the user...")
    else:
        mydb = mysql.connector.connect(
            host = "localhost",
            user = "root",
            passwd = "",
            database = "Authorized_user"
        )
        mycursor = mydb.cursor()
        mycursor.execute("SELECT * from my_table")
        myresult = mycursor.fetchall()
        
        id = 1
        for x in myresult:
            id +=1
        
        sql = "insert into my_table(id,name,age,address) values(%s, %s, %s, %s)"
        val = (id, t1.get(), t2.get(), t3.get())
        mycursor.execute(sql,val)
        mydb.commit()
        
        face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
        def face_cropped(img):
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray, 1.3, 5)
            # scaling factor = 1.3
            # minimum neighbor = 5

            if faces is ():
                return None
            for (x,y,w,h) in faces:
                cropped_face = img[y:y+h,x:x+w]
            return cropped_face

        cap = cv2.VideoCapture(1)
        img_id = 0

        while True:
            ret, frame = cap.read()
            if face_cropped(frame) is not None:
                img_id+=1
                face = cv2.resize(face_cropped(frame), (200,200))
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                file_name_path = "data/user."+str(id)+"."+str(img_id)+".jpg"
                cv2.imwrite(file_name_path, face)
                cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)

                cv2.imshow("Cropped face", face)

            if cv2.waitKey(1)==13 or int(img_id)==200: #13 is the ASCII character of Enter
                break

        cap.release()
        cv2.destroyAllWindows()
        messagebox.showinfo("Result","Generating dataset is completed.....")

b2= tk.Button(canvas1, text="Generate dataset", font=("Algerian",20), bg='pink', fg='black', command = generate_dataset)
b2.place(x=175, y=180)

load2= Image.open("canvas2.png")
photo2 = ImageTk.PhotoImage(load2)

canvas2 =  Canvas(window, width=500, height=250)
canvas2.place(x=5, y=400)
canvas2.create_image(250,125,image=photo2)


def capture_image():
    
    canvas3 = Canvas(window, width=280, height=530)
    canvas3.place(x=515, y=120)
    canvas3.create_image(140, 265, image=photo3)
    
    cap = cv2.VideoCapture(1)
    if cap.isOpened():
        ret, frame = cap.read()
    else:
        ret = False
    
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    # scaling factor = 1.3
    # minimum neighbor = 5

    if faces is ():
        print("No face detected..")
        return None
    else:
        for (x,y,w,h) in faces:
            cropped_face = frame[y:y+h,x:x+w]
        cv2.imwrite("captured_image.jpg", cropped_face)

        load = Image.open("captured_image.jpg")
        photo = ImageTk.PhotoImage(load)

        # Labels can be text or images
        img = Label(canvas3, image=photo, width=200, height=200)
        img.image = photo

        img.place(x=0, y=5)

        cap.release()
    
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")
    
    for (x,y,w,h) in faces:
        id, pred = clf.predict(gray[y:y+h,x:x+w])
        #confidence = int(100*(1-pred/300))

        mydb = mysql.connector.connect(
        host = "localhost",
        user = "root",
        passwd = "",
        database = "Authorized_user"
        )
        mycursor = mydb.cursor()

        mycursor.execute("select name,age,address from my_table where id=" + str(id))
        s = mycursor.fetchall()
        
        a1 = tk.Label(canvas3, text="Name = ", font=("Algerian",20))
        a1.place(x=5, y=250)
        
        b1 = tk.Label(canvas3, text=s[0][0], font=("Algerian",20))
        b1.place(x=100, y=250)
        
        c1 = tk.Label(canvas3, text="Age = ", font=("Algerian",20))
        c1.place(x=5, y=300)
        
        d1 = tk.Label(canvas3, text = s[0][1], font=("Algerian", 20))
        d1.place(x=100, y=300)
        
        e1 = tk.Label(canvas3, text="Address = ", font=("Algerian", 20))
        e1.place(x=5, y=350)
        
        f1 = tk.Label(canvas3, text= s[0][2], font=("Algerian",20))
        f1.place(x=150, y=350)
    
    
b3 = tk.Button(canvas2, text="Capture the image", font=("Algerian",20), bg="gray", fg="black", command=capture_image)
b3.place(x=5, y=50)

def detect_face():
    
    def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)

        for (x,y,w,h) in features:
            cv2.rectangle(img, (x,y), (x+w,y+h), color, 2 )

            id, pred = clf.predict(gray_img[y:y+h,x:x+w])
            confidence = int(100*(1-pred/300))
            
            mydb = mysql.connector.connect(
            host = "localhost",
            user = "root",
            passwd = "",
            database = "Authorized_user"
            )
            mycursor = mydb.cursor()
            
            mycursor.execute("select name from my_table where id=" + str(id))
            s = mycursor.fetchone() #tuple
            
            s  = ''+''.join(s) #string
            
            if confidence>70:
                cv2.putText(img, s , (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            else:
                cv2.putText(img, "UNKNOWN", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)

        return img

    # loading classifier
    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")

    video_capture = cv2.VideoCapture(1)

    while True:
        ret, img = video_capture.read()
        img = draw_boundary(img, faceCascade, 1.3, 6, (255,255,255), "Face", clf)
        cv2.imshow("face Detection", img)

        if cv2.waitKey(1)==13:
            break
    video_capture.release()
    cv2.destroyAllWindows()
    
b4 = tk.Button(canvas2, text="Predict face from live video", font=("Algerian", 20), bg="cyan", fg="black", command=detect_face)
b4.place(x=5, y=150)

load3= Image.open('canvas3.png')
photo3 = ImageTk.PhotoImage(load3)

canvas3 = Canvas(window, width=280, height=530)
canvas3.place(x=515, y=120)
canvas3.create_image(140, 265, image=photo3)

window.geometry("800x680")
window.mainloop()